# Simple 3 steps RNN

(Based on Hands-On Machine Learning of Auriél Géron)

 ### Forward pass without using any of TensorFlow's RNN operations

In [55]:
import numpy as np
import tensorflow as tf

# Size of the input vector at each time step
n_inputs = 3
# Number of recurrent neurons
n_neurons = 5

# Input vectors, one vector for each time step
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
X2 = tf.placeholder(tf.float32, [None, n_inputs])

# Weight matrices
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32, seed=42))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32, seed=42))
# Bias
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

# Output of the recurrent neurons
Y0 = tf.tanh(tf.matmul(X0, Wx) + b) # t=0
Y1 = tf.tanh(tf.matmul(X1, Wx) + tf.matmul(Y0, Wy) + b) # t=1
Y2 = tf.tanh(tf.matmul(X2, Wx) + tf.matmul(Y1, Wy) + b) # t=2

init = tf.global_variables_initializer()

# Mini-batch t=0
X0_batch = np.array([[0, 1, 2], # instance 0
                     [3, 4, 5], # instance 1
                     [6, 7, 8], # instance 2
                     [9, 0, 1]])# instance 3
# Mini-batch t=1
X1_batch = np.array([[9, 8, 7], 
                     [0, 0, 0], 
                     [6, 5, 4], 
                     [3, 2, 1]]) 
# Mini-batch t=2
X2_batch = np.array([[1, 1, 1], 
                     [2, 2, 2], 
                     [3, 3, 3], 
                     [4, 4, 4]])

with tf.Session() as sess:
    init.run()
    feed_dict = {X0: X0_batch, X1: X1_batch, X2: X2_batch}
    Y0_val, Y1_val, Y2_val = sess.run([Y0, Y1, Y2], feed_dict=feed_dict)
    
    print('output at time step 0 \n', Y0_val)
    print('output at time step 1 \n', Y1_val)
    print('output at time step 2 \n', Y2_val)

output at time step 0 
 [[-0.95407802 -0.99674022  0.98660398  0.57785892 -0.98755085]
 [-0.30700076 -0.99999917  0.99999976  0.63324869 -0.99999374]
 [ 0.8457135  -1.          1.          0.68283153 -1.        ]
 [ 1.         -0.99766296  1.         -0.99999917 -0.98221558]]
output at time step 1 
 [[ 0.99872833 -1.          1.         -0.99214447 -1.        ]
 [-0.96752214 -0.9684667   0.99960893 -0.99444956 -0.89819932]
 [ 0.99972194 -0.99999893  1.         -0.99968082 -0.99999726]
 [ 0.99847519 -0.99978107  0.99999833 -0.99886143 -0.11810157]]
output at time step 2 
 [[ 0.58409685 -0.99991119  0.99998206 -0.99647796 -0.11304511]
 [-0.88040298 -0.99998939  0.99997598 -0.87694168 -0.8552171 ]
 [ 0.93676627 -0.99999976  1.         -0.99602008 -0.98977089]
 [ 0.98058808 -1.          1.         -0.91650188 -0.99989784]]


### Forward pass by using dynamic_rnn() 

In [59]:
tf.reset_default_graph()

# Number of steps
n_steps = 3

# Tensor for all inputs at every time step
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # first dimension is the mini-batch size

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

init = tf.global_variables_initializer()

X_batch = np.array([
    #   t=0        t=1        t=2
    [[0, 1, 2], [9, 8, 7], [1, 1, 1]], # instance 0
    [[3, 4, 5], [0, 0, 0], [2, 2, 2]], # instance 1
    [[6, 7, 8], [6, 6, 4], [3, 3, 3]], # instance 2
    [[9, 0, 1], [3, 2, 1], [4, 4, 4]]  # instance 3
])

with tf.Session() as sess:
    init.run()
    feed_dict = {X: X_batch}
    outputs_val, states_val = sess.run([outputs, states], feed_dict=feed_dict)

# instances X time steps X neurons
print('outputs \n', outputs_val)
# state of each instance at the latest time step
print('states \n', states_val)

outputs 
 [[[-0.79243445 -0.07810986 -0.55623132 -0.63593453 -0.39916798]
  [-1.          0.13781041 -0.99933213 -0.99956363  0.99857813]
  [-0.96403474  0.16623263 -0.87753439 -0.87500155  0.86511773]]

 [[-0.99982357  0.07373492 -0.96598244 -0.97201228  0.44489056]
  [-0.58806229 -0.09562692 -0.66055042 -0.75599611  0.71095526]
  [-0.99286997  0.10593171 -0.93090576 -0.89741349  0.86065054]]

 [[-0.99999988  0.22223617 -0.99790233 -0.99819142  0.88079047]
  [-0.99999905  0.63702017 -0.99781162 -0.99711686  0.99896246]
  [-0.99982023  0.39731839 -0.97057396 -0.97906291  0.95586169]]

 [[-0.99984211 -0.520441    0.26326054  0.56000155  0.91094059]
  [-0.98744112  0.37480354 -0.66938627 -0.73687845  0.93673873]
  [-0.99997264  0.35669041 -0.98546028 -0.9888792   0.96933651]]]
states 
 [[-0.96403474  0.16623263 -0.87753439 -0.87500155  0.86511773]
 [-0.99286997  0.10593171 -0.93090576 -0.89741349  0.86065054]
 [-0.99982023  0.39731839 -0.97057396 -0.97906291  0.95586169]
 [-0.99997264  0